<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Anomaly Detection in Robot Welding Process</b>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>Anomaly detection is a must-have when dealing with data, and sensor data are no exception. Despite well-known approaches — from engineering rules to graph and deep learning, it is still a challenge: it is hard to capture all the anomalies, minimize false positives, cope with the diversity of sensors and metrology issues, and deliver actionable insights at a business pace. Reactivity suffers from the volume, diversity, and complexity of time series. Sounds familiar? Let’s see how ClearScape Analytics from Teradata Vantage Cloud helps address these problems.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Spot Welding Quality Assessment</b></p>
<p style = 'font-size:16px;font-family:Arial'>Spot welding is a common technique used for welding car body panels, particularly in the assembly of smaller parts and components. Spot welding involves using a pair of copper electrodes to apply a series of short, high-current welding pulses to the metal, fusing the parts together at specific points or “spots”.</p>

<p style = 'font-size:16px;font-family:Arial'>The automotive industry is known for its high level of automation, and spot welding is one of the most automated processes, heavily reliant on robots to improve efficiency, reduce labour costs, and improve the consistency and quality of the finished product. Poor welding quality is rare, but even so, the consequences of poor quality may not be negligible in terms of rework costs and customer satisfaction, especially when quality issues are detected too late.</p>

<img  src="images/AnomalyWelding.png"/>

<p style = 'font-size:16px;font-family:Arial'>There are many ways to assess the quality of a spot, like tensile or ultrasonic testing to assess the weld strength or the analysis of the welding current measured and recorded during the welding process to name a few. In this demo, we focus on the analysis of the welding current, and more specifically on the resistance, i.e. the voltage-current ratio. The shape of the resistance curve depends on many factors, like a recipe, the nature of the materials, the geometry, and the quality of the electrodes, …</p>



</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1. Start by connecting to the Vantage system.</b></p>


<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
# # '%%capture' suppresses the display of installation steps of the following packages
!pip install tdsense
!pip install imblearn
# !pip install xgboost==1.7.3
#!pip install colorlover
#!pip install teradataml --upgrade teradataml

<p style = 'font-size:18px;font-family:Arial'><b>**RESTART kernel after installing tdsense</b></p>
<p style = 'font-size:16px;font-family:Arial'>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed.  If you uncomment those installs, be sure to restart the kernel after executing those lines. </p>

In [ ]:
import json
import getpass
import pandas as pd
import datetime
from teradataml import *

import tdsense
from tdsense.plot import plotcurves

import numpy as np # linear algebra
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from tdsense.clustering import hierarchy_dendrogram, hierarchy_clustering
%matplotlib inline

from sklearn import datasets
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, f1_score
import time
import pytz


import os
from jdk4py import JAVA, JAVA_HOME, JAVA_VERSION
# Set java path

os.environ['PATH'] = os.environ['PATH'] + os.pathsep + str(JAVA_HOME)
os.environ['PATH'] = os.environ['PATH'] + os.pathsep + str(JAVA)[:-5]

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import defaultdict
import plotly.offline as offline
offline.init_notebook_mode()


from teradataml.dataframe.sql_functions import case
from teradataml import db_drop_table
configure.byom_install_location = "mldb"

display.max_rows = 5
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=AnomalyDetection.ipynb;' UPDATE FOR SESSION; ''')

<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>2. Getting Data for This Demo
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string.</p>   


In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_AnomolyDetection_cloud');"
 # Takes about 50 seconds
# %run -i ../run_procedure.py "call get_data('DEMO_AnomolyDetection_local');"
 # Takes about 3 minutes 30 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Analyze the raw data set</b></p>

<p style = 'font-size:16px;font-family:Arial'>Create a DataFrame to get the data from the table created.</p>



In [ ]:
Sensor_Data = DataFrame(in_schema('DEMO_AnomolyDetection', 'Sensor_Data'))
Sensor_Data

<p style = 'font-size:16px;font-family:Arial'>We create a view with the columns required to get data with proper column names.</p>

In [ ]:
query = f"""
REPLACE VIEW DEMO_AnomolyDetection.V_dataset_01 AS
SELECT
    1 AS PLANT
,   {41} AS ROBOT_ID
,   CAST(A.PARTITION_ID AS BIGINT) AS WELDING_TYPE
,   CAST((DATE '{str(datetime.now()).split(' ')[0]}'  + FLOOR((WELDING_ID-700*WELDING_TYPE)/100))  AS DATE FORMAT 'YYYY-MM-DD') AS WELDING_DAY
,   CAST(A.ID AS BIGINT) AS WELDING_ID
,   CAST(A.X AS INTEGER) AS TIME_MS
,   A.Y AS RESISTANCE
FROM DEMO_AnomolyDetection.Sensor_Data A
"""
eng.execute(query)

In [ ]:
welding_dataset_new = DataFrame(in_schema('DEMO_AnomolyDetection', 'V_dataset_01'))
welding_dataset_new

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>3.1 - Some aggregations and visualization. </b></p>
<p style = 'font-size:16px;font-family:Arial'>We check the total number of weldings done per day</p>

In [ ]:
welding_dataset_new. \
    groupby('WELDING_DAY'). \
    count(distinct=True). \
    sort('WELDING_DAY'). \
    to_pandas(). \
    plot(x='WELDING_DAY',y='count_WELDING_ID', kind='bar')

<p style = 'font-size:16px;font-family:Arial'>We can observe in the above bar chart that the total number of weldings per day are same. </p>

<p style = 'font-size:16px;font-family:Arial'>We will check the histogram based on the minimum and maximum Time for welding.</p>
<p style = 'font-size:16px;font-family:Arial'>A histogram is a better way to assess distribution, to cope with the scalability, it is recommended to compute the histogram bins in-database to leverage the Massively Parallel Architecture of Teradata Vantage. For that, we use the Histogram function of teradataml that pushes down the computations to Vantage.</p>

In [ ]:
welding_duration_ms = welding_dataset_new. \
                        groupby(['PLANT','ROBOT_ID','WELDING_TYPE', 'WELDING_ID']). \
                        agg({'TIME_MS':['min','max','count']})
welding_duration_ms

In [ ]:
from teradataml import Histogram
obj = Histogram(data=welding_duration_ms,
                    target_columns="count_TIME_MS",
                    method_type="Scott")
obj.result.sort('MinValue')

In [ ]:
res = obj.result.sort('MinValue').to_pandas()
res['duration_ms'] = [str(row['MinValue'])+'-'+str(row['MaxValue']) for i,row in res.iterrows()]
res.plot(x='duration_ms',y='CountOfValues',kind='bar', figsize=(15,10), legend=False)

<p style = 'font-size:16px;font-family:Arial'>In the above histogram we can see the bins between the Min and the Max value of the durations and the welding counts.</p> 
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>3.2 - More advanced processing using window functions and delta_t </b></p>

In [ ]:
welding_dataset_new.loc[welding_dataset_new.WELDING_ID == 854]

In [ ]:
plotcurves(welding_dataset_new.loc[welding_dataset_new.WELDING_ID == 854],field='RESISTANCE',row_axis='TIME_MS', series_id='WELDING_ID',select_id=None)

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the variation of the resistance of the welding with respect to time. We see that the most interesting part lies between 40 and 400ms from the start of the curve.</p>

<p style = 'font-size:16px;font-family:Arial'>Next we apply the window function on the resistance to smooth the resistance and taking the mean value.</p>


In [ ]:
# curve smoothing
window_for_smoothing = welding_dataset_new.RESISTANCE.window(
                            partition_columns   = "WELDING_ID",
                            order_columns       = 'TIME_MS',
                            window_start_point  = -15,
                            window_end_point    = 15
)
welding_dataset_smooth = welding_dataset_new.assign(RESISTANCE_SMOOTHED = window_for_smoothing.mean())

In [ ]:
id_curve = 854
single_welding_pandas = welding_dataset_smooth[welding_dataset_smooth.WELDING_ID == id_curve].sort('TIME_MS').to_pandas().reset_index()

fig, ax = plt.subplots(1,1,figsize=(20,7))
single_welding_pandas.plot(x='TIME_MS', y='RESISTANCE', ax=ax)
single_welding_pandas.plot(x='TIME_MS', y='RESISTANCE_SMOOTHED', ax=ax,  xlabel='time (ms)', ylabel=r'resistance  $(m\Omega)$', color='r')

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the variation of the resistance of the welding with respect to time and the smoothed resistance, as shown by the Red line, after applying the window function.</p>

<p style = 'font-size:16px;font-family:Arial'>Next we calculate the derivative by using the lead function and taking the difference of the lead value and the mean value of the resistance.</p>


In [ ]:
# let's compute the lead
window_for_lead = welding_dataset_smooth.RESISTANCE_SMOOTHED.window(
                            partition_columns   = "WELDING_ID",
                            order_columns       = 'TIME_MS')

In [ ]:
welding_dataset_smooth = welding_dataset_smooth.assign(RESISTANCE_SMOOTHED_AFTER = window_for_lead.lead())
welding_dataset_smooth = welding_dataset_smooth.assign(DERIVATIVE = welding_dataset_smooth.RESISTANCE_SMOOTHED_AFTER - welding_dataset_smooth.RESISTANCE_SMOOTHED)

In [ ]:
# it takes approx. 20s
welding_dataset_smooth.sort(['WELDING_ID','TIME_MS'])

In [ ]:
id_curve = 854
single_welding_pandas = welding_dataset_smooth[welding_dataset_smooth.WELDING_ID == id_curve].sort('TIME_MS').to_pandas().reset_index()
single_welding_pandas

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(20,7))
single_welding_pandas.plot(x='TIME_MS', y='RESISTANCE', ax=ax[0])
single_welding_pandas.plot(x='TIME_MS', y='RESISTANCE_SMOOTHED', ax=ax[0],  xlabel='time (ms)', ylabel=r'resistance  $(m\Omega)$', color='r')
single_welding_pandas.plot(x='TIME_MS', y='DERIVATIVE', ax=ax[1],  xlabel='time (ms)', ylabel=r'resistance  $(m\Omega)/ms$', color='r')

<p style = 'font-size:16px;font-family:Arial'>We see that the most interesting part lies between 40 and 400ms from the start of the curve, so we plot only that subset.</p>

<p style = 'font-size:16px;font-family:Arial'>It is hard to assess the diversity of curve shapes in this plot since many of them are superimposed. However, we see in the middle of the picture a sharp drop that looks unusual. Moreover, we guess that there are shifts in time and height.</p>
<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>4. Feature Engineering

In [ ]:
welding_dataset_new.columns

<p style = 'font-size:16px;font-family:Arial'>We will create a feature table by using different functions on the Resistance column. Valid values for functions are: 'count', 'sum', 'min', 'max', 'mean', 'std', 'percentile', 'unique','median', 'var', 'skew', 'kurtosis'. </p>

In [ ]:
features = welding_dataset_new.loc[welding_dataset_new.TIME_MS > 20,:]. \
        groupby(welding_dataset_new.columns[0:5]). \
        agg({
            'TIME_MS':['min','max'],
            'RESISTANCE':['count', 'sum', 'min', 'max', 'mean', 'std', 'percentile', 'unique','median', 'var','skew','kurtosis']
        })
features

<p style = 'font-size:18px;font-family:Arial'>Store the features in a table in database.

In [ ]:
features.to_sql(
    table_name  = 'welding_features_01',
    schema_name = 'demo_user',
    primary_index= 'WELDING_ID',
    if_exists = 'replace'
)

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Anomaly Detection on Sensor Data</b></p>
    
<p style = 'font-size:16px;font-family:Arial'>Let's start by getting the feature columns from the features tables</p>   

In [ ]:
feature_names = features.columns[7::]
feature_names

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>5.1 Clustering by curve shape</b></p>
<p style = 'font-size:16px;font-family:Arial'>To cluster time series by shapes, we will use the Dynamic Time Warping (DTW) distance that measures the similarity between two time series. This distance is well adapted to this kind of problem since it provides robustness to shifts in time and height.</p>

<p style = 'font-size:16px;font-family:Arial'>Distance Matrix in-database Computations</p>

<p style = 'font-size:16px;font-family:Arial'>The ClearScape Analytics DTW function computes at scale distances between one reference curve to a set of curves, a many-to-one approach. ClearScape Analytics offers in database dynamic time warping function, callable in SQL as TD_DTW. This function computes at scale the DTW distances between one reference curve to a set of curves, a many-to-one approach. We want to compute the distance matrix of our subset, i.e. the DTW distance between each curve. The distance matrix is symmetric, since the DTW is, hence we only need to compute the triangular matrix. We wrapped this computation in the tdsense package that calls the TD_DTW function and iterates on the matrix row to compute and store the whole triangular distance matrix in a table.</p>

In [ ]:
overview = welding_dataset_new.groupby('WELDING_DAY').count(distinct=True)
dates = list(overview.to_pandas().reset_index()['WELDING_DAY'].values.astype('str'))
dates

In [ ]:
subset = welding_dataset_new[ \
                 (welding_dataset_new['PLANT'] == 1) & \
                 (welding_dataset_new['ROBOT_ID'] == 41) & \
                 (welding_dataset_new['WELDING_TYPE'] in (8,9)) & \
                 (welding_dataset_new['WELDING_DAY'].isin(dates)) \
                ]

In [ ]:
subset_zoom = subset[(subset.TIME_MS < 400) & (subset.TIME_MS > 40)]
subset_zoom.shape

<p style = 'font-size:16px;font-family:Arial'>Since this is a very small(4 amps) system, the below computation takes around more than 2 hours for 350k rows and so we have pre calculated it and stored in the table in database.</p>

<p style = 'font-size:16px;font-family:Arial'><i>**In case you still want to compute the matrix please set the If part of the below code to <b>True</b> instead of <b>False</b></i></p>

In [ ]:
if False:
    dtw_matrix = dtw_distance_matrix_computation2(subset_zoom,field='RESISTANCE',
                                     table_name=dtw_result_table,
                                     schema_name = Param['database'],
                                     row_axis='TIME_MS',
                                     series_id = 'WELDING_ID')
else:
    dtw_matrix = DataFrame(in_schema('DEMO_AnomolyDetection','DTW_Matrix'))

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>5.2 Hierarchical clustering with Scipy</p>

<p style = 'font-size:16px;font-family:Arial'>Now the distance matrix is available, we can perform the clustering. Here we will use the open-source package Scipy and its cluster.hierarchy modules, that have been used in a tdsense for convenience.</p>

In [ ]:
dtw_matrix_loc = dtw_matrix.sort(columns=['WELDING_ID_2','WELDING_ID_1']).to_pandas(all_rows=True)
dtw_matrix_loc

In [ ]:

linked, labelList = hierarchy_dendrogram(dtw_matrix_loc, cluster_distance = 'ward')

In [ ]:
cluster = hierarchy_clustering(linked, labelList, n_clusters=6)
cluster.head()

<p style = 'font-size:16px;font-family:Arial'>And if we plot the curves per cluster, we spot the curves with a sharp drop(cluster 2) and these are the curves we are interested in, i.e. the curve exhibiting the anomaly we are looking for. We note also the other clusters are looking more or less similar.</p>

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(20,10))
colors = cluster[['cluster','leaves_color_list']].copy().drop_duplicates()
for k in range(6):
    plt.subplot(2,3,k+1)
    img = plotcurves( subset_zoom,
                      field='RESISTANCE',
                      row_axis='TIME_MS',
                      series_id='WELDING_ID',
                      select_id=list(cluster[cluster.cluster ==k].CURVE_ID.values),
                      noplot=True)
    plt.imshow(img)
    plt.title('cluster : ' +str(k) + '\n' + str(cluster.groupby('cluster').count()['CURVE_ID'][k]) + ' obs.',fontdict = {'fontsize' : 10, 'color':colors.leaves_color_list.values[k]})
    plt.axis('off')

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>5.3 Create the anomaly dataset</b></p>
<p style = 'font-size:16px;font-family:Arial'>Now we create a table containing the anomaly flag that will be the target of a supervised machine learning model or a relevant KPI to monitor in production dashboards.</p>



In [ ]:
target = cluster.copy().drop('leaves_color_list',axis=1)
target = target[target.cluster.isin([1,2])]
target['WELDING_ID'] = target['CURVE_ID']
target['anomaly'] = 0
target.loc[target.cluster==2,'anomaly'] = 1
target.drop(['cluster','CURVE_ID'],axis=1, inplace=True)
target.groupby('anomaly').count().plot(y='WELDING_ID',kind='bar',figsize=(10,10))
copy_to_sql( target,
                  schema_name='demo_user',
                  table_name = 'Anomoly_Target',
                  if_exists='replace',
                  primary_index='WELDING_ID')

In [ ]:
anomalies = DataFrame(in_schema('demo_user', 'Anomoly_Target'))
anomalies

<p style = 'font-size:16px;font-family:Arial'>The above anomaly data has the welding ID and the anomaly flag.</p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>5.4 Build the analytical dataset </b></p>

<p style = 'font-size:16px;font-family:Arial'>We prepare the analytical dataset by joining the feature table with the anomaly table using the Welding ID so that we get the anomalies for the weldings.</p>

In [ ]:
ADS = features[['WELDING_ID']+feature_names].join(other=anomalies, how='inner', on='WELDING_ID=WELDING_ID',rsuffix='r',lsuffix='l')
ADS = ADS.assign(WELDING_ID=ADS.l_WELDING_ID).drop(['l_WELDING_ID','r_WELDING_ID'],axis=1).select(['WELDING_ID']+feature_names+['anomaly'])
ADS

In [ ]:
ADS.shape

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Build the model </b></p>
<p style = 'font-size:16px;font-family:Arial'>We have datasets in which different columns have different units – like one column can be in kilograms, while another column can be in centimeters. If we feed these features to the model as is, there is every chance that one feature will influence the result more due to its value than the others. But this doesn’t necessarily mean it is more important as a predictor. So, to give importance to all the features we need feature scaling.</p>
    
<p style = 'font-size:16px;font-family:Arial'>Here, we apply the Standard scale and transform functions which are ScaleFit and ScaleTransform functions in Vantage. ScaleFit() function outputs statistics to input to ScaleTransform() function, which scales specified input DataFrame columns.</p> 

In [ ]:
from teradataml import ScaleFit , ScaleTransform
scaler = ScaleFit(
                    data=ADS,
                    target_columns=feature_names,
                    scale_method="STD",
                    global_scale=False)

In [ ]:
ADS_scaled = ScaleTransform(data=ADS,
                         object=scaler.output,
                         accumulate="anomaly").result
ADS_scaled

In [ ]:
scaler.output

In [ ]:
scaler.output.to_sql(schema_name='demo_user',table_name='scaler_anomaly',if_exists='replace')

In [ ]:
df = ADS_scaled.to_pandas()
df

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.1 Create a model file using the python libraries.</b></p>

<p style = 'font-size:16px;font-family:Arial'>A problem with imbalanced classification is that there are too few examples of the minority class for a model to effectively learn the decision boundary. One way to solve this problem is to oversample the examples in the minority class. the most widely used approach to synthesizing new examples is called the Synthetic Minority Oversampling Technique, or SMOTE for short. SMOTE works by selecting examples that are close in the feature space, drawing a line between the examples in the feature space and drawing a new sample at a point along that line.</p>

<p style = 'font-size:16px;font-family:Arial'>Then we use the RandomForestClassifier to create the model. A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The Random forest classifier creates a set of decision trees from a randomly selected subset of the training set. It is basically a set of decision trees (DT) from a randomly selected subset of the training set and then It collects the votes from different decision trees to decide the final prediction.</p>

In [ ]:
X_train = df[feature_names]
y_train = df['anomaly']

In [ ]:
# Balance the training set using SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


# Create a random forest classifier
model = RandomForestClassifier(n_estimators=10,max_depth= 3, random_state=42)

# Create a pipeline that includes the SMOTE transformer and the model
pipeline = PMMLPipeline([ ('model', model)])


In [ ]:
# Train the pipeline
start = time.time()
pipeline.fit(X_train, y_train)
end = time.time()
print('duration : ', end-start, 's')

In [ ]:
# make predictions on the training set
y_train_pred = pipeline.predict(X_train)

# calculate and print the accuracy score
acc = accuracy_score(y_train, y_train_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

# calculate and print precision, AUC and F1-score
prec = precision_score(y_train, y_train_pred)
print("Precision: {:.2f}%".format(prec * 100))

# calculate AUC, AUC requires probability for positive class
prob = pipeline.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, prob)
print("AUC: {:.2f}%".format(auc * 100))

f1 = f1_score(y_train, y_train_pred)
print("F1-Score: {:.2f}%".format(f1 * 100))

In [ ]:
sklearn2pmml(pipeline, "my_model.pmml", with_repr = True)

In [ ]:
additional_columns = {"Description": type("RandomForestClassifier model"),
                              "UserId": type('demo_user'),
                              "ProductionReady": False,
                              "ModelAccuracy": float(acc),
                              "ModelPrecision": prec,
                              "ModelAUC": auc,
                              "Modelf1Score": f1,
                              "ModelSavedTime": str(datetime.now(tz=pytz.UTC)),
                              "ModelGeneratedTime": end-start,
                              "sklearnVersion": sklearn.__version__
                             }
for k in additional_columns.keys():
    print(type(additional_columns[k]))

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.2 Save the model file</b></p>

In [ ]:
try:
    save_byom(model_id = 'model_anomaly1',
          model_file = 'my_model.pmml',
          table_name = 'BYOM_PMMLMODELS_REPOSITORY',
          schema_name = 'demo_user',
          additional_columns={"Description": "RandomForestClassifier model",
                              "UserId": 'demo_user',
                              "ProductionReady": False,
                              "ModelAccuracy": float(acc),
                              "ModelPrecision": float(prec),
                              "ModelAUC": float(auc),
                              "Modelf1Score": float(f1),
                              "ModelSavedTime": str(datetime.now(tz=pytz.UTC)),
                              "ModelGeneratedTime": float(end-start),
                              "sklearnVersion": sklearn.__version__
                             }
            )
except Exception as e: 
    # if our model exists, delete and rewrite if str(e.args).find('TDML_2200') >= 1: 
    delete_byom(model_id = 'model_anomaly1', table_name = 'BYOM_PMMLMODELS_REPOSITORY',schema_name = 'demo_user') 
    save_byom(model_id = 'model_anomaly1',
          model_file = 'my_model.pmml',
          table_name = 'BYOM_PMMLMODELS_REPOSITORY',
          schema_name = 'demo_user',
          additional_columns={"Description": "RandomForestClassifier model",
                              "UserId": 'demo_user',
                              "ProductionReady": False,
                              "ModelAccuracy": float(acc),
                              "ModelPrecision": float(prec),
                              "ModelAUC": float(auc),
                              "Modelf1Score": float(f1),
                              "ModelSavedTime": str(datetime.now(tz=pytz.UTC)),
                              "ModelGeneratedTime": float(end-start),
                              "sklearnVersion": sklearn.__version__
                             }
            )
    pass 
else: 
    raise    

<p style = 'font-size:16px;font-family:Arial'>The model file is saved as can be seen in the left pane(file editor).</p>

<p style = 'font-size:16px;font-family:Arial'>We create new scaled data to apply this model and predict data. </p>

In [ ]:
newdata = features[['WELDING_ID']+feature_names].join(other=anomalies, how='inner', on='WELDING_ID=WELDING_ID',rsuffix='r',lsuffix='l')
newdata = newdata.assign(WELDING_ID=newdata.l_WELDING_ID).drop(['l_WELDING_ID','r_WELDING_ID'],axis=1).select(['WELDING_ID']+feature_names+['anomaly'])
newdata

In [ ]:
newdata.shape

In [ ]:
newdata_scaled = ScaleTransform(data=newdata,
                         object=DataFrame(in_schema('demo_user','scaler_anomaly')),
                         accumulate=["WELDING_ID","anomaly"]).result
newdata_scaled

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.3 Retrieve the model file and use it to predict</b></p>
<p style = 'font-size:16px;font-family:Arial'>We use the PMMLPredict function from the teradataml library to predict the anomalies.</p>
<p style = 'font-size:16px;font-family:Arial'>Predictive Model Markup Language (PMML) is an XML-based standard established by the Data Mining Group (DMG) for defining statistical and data-mining models. PMML models can be shared between PMML-compliant platforms and across organizations so that business analysts and developers are unified in designing, analyzing, and implementing PMML-based assets and services.</p>

In [ ]:
from teradataml import PMMLPredict
modeldata_anomoly = retrieve_byom("model_anomaly1", table_name="BYOM_PMMLMODELS_REPOSITORY")
result=PMMLPredict(
                modeldata = modeldata_anomoly,
                newdata = newdata_scaled,
                accumulate = ['WELDING_ID'],
                overwrite_cached_models = '*'
                )
result.result

In [ ]:
query_expand_json = f"""
SELECT 
    WELDING_ID
      ,   "probability(0)" as proba_0
,   "probability(1)" as proba_1
FROM TD_JSONSHRED(
    ON (SELECT WELDING_ID, json_report FROM  {result.result._table_name})
    USING
    ROWEXPR('')
    COLEXPR('probability(1)','probability(0)')
    RETURNTYPES('FLOAT','FLOAT')
    ) t
"""
sanitized_results = DataFrame.from_query(query_expand_json)
sanitized_results

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>7. Decision Forest </b></p>
 
<p style = 'font-size:16px;font-family:Arial'>We will now use the DecisionForest model to predict the anomalies. A decision forest is a generic term to describe models made of multiple decision trees. The prediction of a decision forest is the aggregation of the predictions of its decision trees. The implementation of this aggregation depends on the algorithm used to train the decision forest. The goal of using a Decision Tree is to create a training model that can use to predict the class or value of the target variable by learning simple decision rules inferred from prior data(training data).</p>

<p style = 'font-size:16px;font-family:Arial'>We start by creating a subset for the most interesting part lies between 40 and 400ms from the start of the curve.</p>
        
        

In [ ]:
DF_curves_zoom = welding_dataset_new[(welding_dataset_new.TIME_MS > 40) & (welding_dataset_new.TIME_MS < 400) ]
DF_curves_zoom

<p style = 'font-size:16px;font-family:Arial'>Create the various features by using the window function on the Resistance and taking the difference between the previous and current resistance based on time. Features will be created by using the aggregation function on this resistance and the difference of the resistance.</p>
        

In [ ]:
DF_curves_zoom = DF_curves_zoom.assign(
    resistance_diff = DF_curves_zoom.RESISTANCE 
                        - DF_curves_zoom.RESISTANCE.window(
                                partition_columns=['WELDING_ID'],
                                order_columns=["TIME_MS"]
                            ).lag(1)
)
# DF_curves_zoom[DF_curves_zoom.WELDING_ID==138].sort("TIME_MS")

In [ ]:
DF_features = DF_curves_zoom.groupby("WELDING_ID").agg({
    'RESISTANCE':['sum', 'min', 'max', 'mean', 'std', 'var','skew','kurtosis'],
    'resistance_diff':['min']
})
DF_features

In [ ]:
feature_names = DF_features.columns[1:]
feature_names

<br>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.1 Build the anaytical dataset.</b></p>
<p style = 'font-size:18px;font-family:Arial'>The anaytical dataset is created by joining the anomaly table created above and the dataset with the features created.</p>

In [ ]:
DF_target = DataFrame(in_schema('demo_user', 'Anomoly_Target'))

In [ ]:
DF_ADS_train = DF_features[['WELDING_ID']+feature_names].join(
    other=DF_target, how='inner', on='WELDING_ID=WELDING_ID',rsuffix='r',lsuffix='l')
DF_ADS_train = DF_ADS_train.assign(WELDING_ID=DF_ADS_train.l_WELDING_ID
                                  ).drop(['l_WELDING_ID','r_WELDING_ID'],axis=1
                                        ).select(['WELDING_ID']+feature_names+['anomaly']
                                                ).assign(anomaly_int = DF_ADS_train.anomaly.cast(INTEGER()))
DF_ADS_train

In [ ]:
DF_ADS_score = DF_features[['WELDING_ID']+feature_names]\
                            [DF_features.WELDING_ID>800]
DF_ADS_score

<p style = 'font-size:18px;font-family:Arial'>We store these training and scoring datasets into Vantage to be used by the In-DB functions.</p>

In [ ]:
DF_ADS_train.to_sql(
    table_name  = 'ADS_train_data',
    schema_name = 'demo_user',
    primary_index= 'WELDING_ID',
    if_exists = 'replace'
)

In [ ]:
DF_ADS_score.to_sql(
    table_name  = 'ADS_test_data',
    schema_name = 'demo_user',
    primary_index= 'WELDING_ID',
    if_exists = 'replace'
)

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.2 Train Decision Forest</b></p>
<p style = 'font-size:16px;font-family:Arial'>The <a href = 'https://docs.teradata.com/search/all?query=TD_DecisionForest&content-lang=en-US'>TD_DecisionForest</a> is an ensemble algorithm used for classification and regression predictive modeling problems. It is an extension of bootstrap aggregation (bagging) of decision trees. </p>

<p style = 'font-size:16px;font-family:Arial'>This function takes the training data as input, as well as the following function parameters</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>InputColumns; list or range of columns used as features (we used an ordinal reference of columns 2:217)</li>
        <li>ResponseColumn; the dependent or target value (we used “class”, the first column)</li>
        <li>TreeType; either CLASSIFICATION or REGRESSION</li>
    <li>Other hyperparameter values detailed in the documentation</li>
        </ul>

In [ ]:
query = '''Create table DF_train as (
SELECT * FROM TD_DecisionForest (
ON ADS_train_data AS INPUTTABLE partition by ANY
USING
  ResponseColumn('anomaly_int')
InputColumns('sum_RESISTANCE', 'min_RESISTANCE', 'max_RESISTANCE', 'mean_RESISTANCE', 'std_RESISTANCE', 'var_RESISTANCE', 'skew_RESISTANCE',
 'kurtosis_RESISTANCE', 'min_resistance_diff')
MaxDepth(16)
MinNodeSize(1)
NumTrees(8)
ModelType('CLASSIFICATION')
Seed(3)
Mtry(1)
MtrySeed(3)
) AS dt
) with data;
'''
 
try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE DF_train;')
    eng.execute(query)

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.3 Predict and Evaluate Decision Forest model</b></p>
<p style = 'font-size:16px;font-family:Arial'>Execute a testing prediction using the split data above.  Evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> SQL Function.</p>


<ol style = 'font-size:16px;font-family:Arial'>
    <li>Execute <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Model-Scoring-Functions/DecisionForestPredict'>DecisionForestPredict</a> using the model built above</li>
    <li>Execute <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> and pass the actual classification and the predicted value</li>
</ol>

In [ ]:
query = '''
Create table DF_Predict as (
SELECT * FROM TD_DecisionForestPredict (
ON ADS_train_data AS InputTable PARTITION BY ANY
ON DF_Train AS ModelTable DIMENSION
USING
  IdColumn ('WELDING_ID')
  Accumulate ('anomaly_int')
  Detailed('false')
  OutputProb ('true')
  Responses ('0','1')
) AS dt) with data;'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE DF_Predict;')
    eng.execute(query)

In [ ]:
df_predict= DataFrame(in_schema('demo_user', 'DF_Predict'))
df_predict

In [ ]:
query = '''
SELECT * FROM TD_ClassificationEvaluator(
   ON DF_predict AS InputTable
   --cast(target as VARCHAR(32000) CHARACTER SET UNICODE NOT CASESPECIFIC) as target from DF_predict_test) AS InputTable
   OUT VOLATILE TABLE OutputTable(additional_metrics_test)
   USING
   ObservationColumn('anomaly_int')
   PredictionColumn('prediction')
   Labels(0,1)
) AS dt;
'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE additional_metrics_test;')
    eng.execute(query)

# pd.read_sql('SELECT * FROM additional_metrics_test', eng)
df_metrics= DataFrame(in_schema('demo_user', 'additional_metrics_test'))
df_metrics

In [ ]:
DF_eval=DataFrame(in_schema('demo_user', 'DF_Predict'))

In [ ]:
# Probability can be used to further discriminate!
DF_eval.groupby(["anomaly_int","prediction"]).agg(
    {"prob_0":["mean","std"]}
)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.4 Show AUC-ROC Curve for DecisionForestPredict</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <a href = 'https://docs.teradata.com/search/all?query=TD_ROC&content-lang=en-US'>ROC</a> curve shows the performance of a binary classification model as its discrimination threshold varies. For a range of thresholds, the curve plots the true positive rate against false-positive rate.</p>

<p style = 'font-size:16px;font-family:Arial'>This function accepts a set of prediction-actual pairs as input and calculates the following values for a range of discrimination thresholds.</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>True-positive rate (TPR)</li>
        <li>False-positive rate (FPR)</li>
        <li>The area under the ROC curve (AUC)</li>
        <li>Gini coefficient</li>
        <li>Other details are mentioned in the documentation</li>
    </ul>

In [ ]:
from teradataml import ROC 
roc_obj = ROC(DF_eval, 
                    probability_column = "prob_1",
                    observation_column = "anomaly_int",
                    positive_class="1"
                    )

In [ ]:
roc_data = roc_obj.output_data.to_pandas().sort_values("fpr", ascending=True)
roc_data.tail(10)

In [ ]:
auc = roc_obj.result.to_pandas().reset_index().iloc[0,0]
auc

In [ ]:
# Plot of a ROC curve for DecisionForest

plt.figure()
plt.plot(roc_data.fpr, roc_data.tpr, label='ROC curve (area = %0.2f)' % auc, drawstyle='steps')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate',fontsize=12)
plt.ylabel('True Positive Rate',fontsize=12)
plt.title('Receiver operating characteristic curve for DecisionForest',fontsize=14)
plt.legend(loc="lower right",fontsize=10)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>It looks like the optimal threshold for prob_1 is around 0.2 (see table roc_data).</p>

In [ ]:
#TODO: Set meaningful threshold, where true positive rate is above 80% and false positive rate below 20%
THRESHOLD = 0.2

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.5 Score new Data</b></p>

In [ ]:
query = '''
Create table DF_Predict_test as (
SELECT * FROM TD_DecisionForestPredict (
ON ADS_test_data AS InputTable PARTITION BY ANY
ON DF_Train AS ModelTable DIMENSION
USING
  IdColumn ('WELDING_ID')
  Detailed('false')
  OutputProb ('true')
  Responses ('0','1')
) AS dt) with data;'''

try:
    eng.execute(query)
except:
    eng.execute('DROP TABLE DF_Predict_test;')
    eng.execute(query)

In [ ]:
df_predict_test= DataFrame(in_schema('demo_user', 'DF_Predict_test'))
df_predict_test

In [ ]:
DF_scored=DataFrame(in_schema('demo_user', 'DF_Predict_test'))

In [ ]:
prob_1 = DF_scored.prob_1
DF_scored = DF_scored.assign(above_threshold  = case([(prob_1>THRESHOLD, 1 )],
                                         else_ = 0))
DF_scored

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.6 Validate data visually</b></p>

In [ ]:
welding_ids_NiO = DF_scored[DF_scored.above_threshold==1]\
    .select(["WELDING_ID"]).head(30).to_pandas().reset_index()\
    ["WELDING_ID"].values.tolist()
welding_ids_iO = DF_scored[DF_scored.above_threshold==0]\
    .select(["WELDING_ID"]).head(30).to_pandas().reset_index()\
    ["WELDING_ID"].values.tolist()

In [ ]:
DF_scored_NiO = DF_curves_zoom[DF_curves_zoom.WELDING_ID.isin(welding_ids_NiO)]


In [ ]:
DF_scored_iO = DF_curves_zoom[DF_curves_zoom.WELDING_ID.isin(welding_ids_iO)]


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))
plt.subplot(1,2,1)
img = plotcurves(DF_scored_NiO,field='RESISTANCE',row_axis='TIME_MS', series_id='WELDING_ID',select_id=None,noplot=True, color="r")
plt.imshow(img)
plt.title('NiO',fontdict = {'fontsize' : 10})
plt.axis('off')
plt.subplot(1,2,2)
img = plotcurves(DF_scored_iO,field='RESISTANCE',row_axis='TIME_MS', series_id='WELDING_ID',select_id=None ,noplot=True, color="b")
plt.imshow(img)
plt.title('iO',fontdict = {'fontsize' : 10})
plt.axis('off')
plt.suptitle('Visual Inspection of Scored Data')

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>8. Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have seen an end-to-end exploration process for labeling anomalous time series using ClearScape Analytics on Teradata Vantage. Thanks to the in-database capabilities offered by Teradata Vantage with ClearScape Analytics, we were able to run this exploration with the smallest notebook instance.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>9. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>

In [ ]:
tables = ['welding_features_01', 'scaler_anomaly', 'ADS_train_data', 'ADS_test_data',
          'DF_train', 'DF_Predict', 'DF_Predict_test',
          'additional_metrics_test']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    db_drop_table(table_name=table, schema_name='demo_user')
    # Construct the drop table SQL statement
    # drop_table_sql = f"DROP TABLE {table};"
    # Execute the drop table command
    # eng.execute(drop_table_sql)

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_AnomolyDetection');" 
#Takes 40 seconds

In [ ]:
remove_context()

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>